**Tripadvisor 국내 투어 프로그램 리뷰 데이터 분석을 통한 랜선투어 상품 개발**
- 목적 : Tours & Tickets 메뉴에 등록된 국내 투어 프로그램 리뷰 데이터를 분석하여 오프라인 경험을 최대한 살릴 수 있는 랜선투어 상품 개발
- 단계 :<br>
1) 리뷰수, 평균평점을 기준으로 Top N 인기 프로그램 선정<br>
2) 선정 프로그램의 리뷰 데이터 크롤링<br>
3) 리뷰 내용을 평점별로 구분하여 분석 (높은 평점 vs 낮은 평점)<br>
4) 분석 내용을 기반으로 국내 랜선투어 프로그램 개발

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [2]:
driver = webdriver.Chrome('./chromedriver.exe')

In [3]:
def tour_search(program):
    url = f'https://www.tripadvisor.com/AttractionProductReview-g294197-d12521956-{program}-Seoul.html'
    return url

In [4]:
program = 'Seoul Private Flexible Adventure Tour'
url = tour_search(program)
driver.get(url)
driver.implicitly_wait(10)

In [5]:
def get_reviews(driver):
    # 1) Read more 클릭한 페이지 html정보 가져오기
    try:
        driver.find_element_by_xpath("//div[@data-automation='reviewReadMoreCTA_0']").click()
        driver.find_element_by_xpath("//div[@data-automation='reviewReadMoreCTA_1']").click()
        driver.find_element_by_xpath("//div[@data-automation='reviewReadMoreCTA_2']").click()
        driver.find_element_by_xpath("//div[@data-automation='reviewReadMoreCTA_3']").click()
        driver.find_element_by_xpath("//div[@data-automation='reviewReadMoreCTA_4']").click()
        time.sleep(3)
        
    except:
        pass
    
    results = driver.page_source

    # 2) 본문 내용 가져오기
    soup = BeautifulSoup(results, 'lxml')
    review_pg1_open = soup.select_one('[data-automation = "reviewList"]')
    
    result_pg = []
    
    for review in review_pg1_open: 
        tour_title = soup.select('span.IKwHbf8J')[0].text
        review_title = review.select('a._1xr4qUQI > span')[0].text
        review_comment = review.select('q._2vmgOjMl > span')[0].text
        result = [tour_title, review_title, review_comment]
        result_pg.append(result)
    
    df = pd.DataFrame(result_pg)
    df.columns = [['tour_title', 'review_title', 'review_comment']]
    
    return df

def next_reviews(driver):

    result_all_pg = pd.DataFrame()
    
    for page in range(0,2):
        pg_review =get_reviews(driver)
        time.sleep(5)
        result_all_pg = pd.concat([result_all_pg, pg_review], axis = 0)

        try:
            driver.find_element_by_xpath("//a[@data-automation='pageLink_next']").click()
            time.sleep(3)
            page += 1
            
        except:
            pass
            
    return result_all_pg

In [6]:
reviews = next_reviews(driver)
reviews

,tour_title,review_title,review_comment
0,Seoul Private Flexible Adventure Tour,Best Tour Guide in Korea!,My family and I travelled in Korea this Winter...
1,Seoul Private Flexible Adventure Tour,Perfect Trip!,We planned a private tour just days before our...
2,Seoul Private Flexible Adventure Tour,"Thank you note to our tour guide , Mr Don Lee.",My fanily n I would like thank Mr Don Lee for ...
3,Seoul Private Flexible Adventure Tour,"Good, but slightly below expectations",I guess our expectations were slightly hyped b...
4,Seoul Private Flexible Adventure Tour,Tour guide ok but not great.,Our guide was raised in the US (Alabama to be ...
0,Seoul Private Flexible Adventure Tour,2 day guide,My wife and I visited Korea for the first time...
1,Seoul Private Flexible Adventure Tour,One of the best tours ever,"Our tour guide, Don, was just wonderful. My fr..."
2,Seoul Private Flexible Adventure Tour,Had an amazing stay in South Korea.,You forget stress when traveling. Sometimes we...
3,Seoul Private Flexible Adventure Tour,Great tour,We had a great 2 day tour. It was our first tr...
4,Seoul Private Flexible Adventure Tour,A great trip !,A very entertaining and valuable English-speak...


In [7]:
reviews.to_excel('review_pg2.xlsx')

**향후 과제**
- 1페이지 리뷰만 잘리는 문제 해결
- csv파일로 저장시 오류나지 않는 방법 확인하기